# 🧩 Phase 3: GEDIG Maze - Large Scale GPU Experiment

このノートブックでは、GPU最適化されたColab環境で、GEDIG（Graph Edit Distance + Information Gain）迷路解法アルゴリズムの大規模実験を行います。

## 🎯 実験目標
- **試行回数削減**: 60%削減
- **収束速度**: 3倍高速収束  
- **成功率**: 95%達成

複数のアルゴリズム（A*, Dijkstra, Q-Learning, 遺伝的アルゴリズム、SlimeMold_GEDIG）を比較し、GEDIG手法の優位性を実証します。

## 🔧 統一セットアップ
InsightSpike-AI統一セットアップスクリプトでGEDIG迷路実験環境を準備します。

In [ ]:
# Phase3用統一セットアップ実行
import os
import subprocess
import sys

# InsightSpike-AIリポジトリの準備
if not os.path.exists('/content/InsightSpike-AI'):
    print("📂 InsightSpike-AIリポジトリをクローン中...")
    !git clone https://github.com/miyauchi-kazuyoshi/InsightSpike-AI.git /content/InsightSpike-AI
    os.chdir('/content/InsightSpike-AI')
else:
    print("📂 リポジトリは既に存在します")
    os.chdir('/content/InsightSpike-AI')

print("🚀 Phase3用統一セットアップ実行中...")

try:
    # 統一セットアップスクリプト実行
    !chmod +x scripts/colab/setup_unified.sh
    result = subprocess.run(['bash', 'scripts/colab/setup_unified.sh'], 
                          capture_output=True, text=True, cwd='/content/InsightSpike-AI')
    
    print("📋 統一セットアップ出力:")
    print(result.stdout)
    
    if result.stderr:
        print("⚠️ 警告:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✅ 統一セットアップ完了!")
    else:
        print("❌ セットアップでエラーが発生しました")

except Exception as e:
    print(f"⚠️ 統一セットアップ失敗: {e}")

# Phase3専用ライブラリ追加インストール
print("📦 Phase3専用GEDIG迷路ライブラリを追加インストール中...")
phase3_specific_libraries = [
    "networkx",
    "scipy",
    "scikit-learn", 
    "imageio",
    "pillow",
    "plotly",
    "matplotlib",
    "seaborn",
    "torch",
    "numpy",
    "tqdm"
]

for lib in phase3_specific_libraries:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "--upgrade", "-q"])
        print(f"✅ {lib}")
    except Exception as e:
        print(f"⚠️ {lib} インストール失敗: {e}")

# Python パス設定
sys.path.insert(0, '/content/InsightSpike-AI/src')
sys.path.insert(0, '/content/InsightSpike-AI/experiments_colab/shared')

# GPU環境確認
import torch
import numpy as np
import random

print(f"\n🎮 GPU環境:")
print(f"  CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU名: {torch.cuda.get_device_name(0)}")
    print(f"  GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# ランダムシード設定（再現性のため）
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("🎲 ランダムシード設定完了")

print("🎯 Phase3 GEDIG迷路実験環境セットアップ完了!")

## 🔧 環境セットアップ
GPU最適化されたColab環境でGEDIG迷路実験の準備を行います。

In [ ]:
# GPU環境確認とライブラリインストール
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import sys
import time
import random
from collections import deque, defaultdict
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Optional
import heapq
import copy

print("🔍 Colab環境チェック")
print("=" * 50)

# GPU確認
if torch.cuda.is_available():
    print(f"✅ GPU利用可能: {torch.cuda.get_device_name(0)}")
    print(f"📊 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
else:
    print("❌ GPU利用不可 - CPUで実行します")
    device = torch.device('cpu')

print(f"🔧 PyTorch Version: {torch.__version__}")
print(f"🎯 使用デバイス: {device}")

# 必要なライブラリのインストール
libraries = [
    "networkx",
    "scipy", 
    "scikit-learn",
    "imageio",
    "pillow",
    "plotly"
]

print("\n📦 ライブラリインストール中...")
for lib in libraries:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"✅ {lib}")
    except Exception as e:
        print(f"⚠️ {lib} インストール失敗: {e}")

# 追加のインポート
import networkx as nx
from scipy.spatial.distance import euclidean
from sklearn.metrics import adjusted_rand_score

print("\n🚀 環境セットアップ完了!")

# ランダムシード設定（再現性のため）
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("🎲 ランダムシード設定完了")

## 🗺️ 迷路環境とアルゴリズム実装
多様な迷路環境と5つの解法アルゴリズムを実装します。

In [ ]:
# 迷路環境クラスと基本アルゴリズム実装

@dataclass
class MazeResult:
    algorithm: str
    maze_type: str
    maze_size: Tuple[int, int]
    success: bool
    steps: int
    path_length: int
    execution_time: float
    path: List[Tuple[int, int]]
    solution_quality: float = 0.0

class Maze:
    """迷路環境クラス"""
    
    def __init__(self, width: int, height: int, maze_type: str = "simple"):
        self.width = width
        self.height = height
        self.maze_type = maze_type
        self.grid = np.ones((height, width), dtype=int)  # 1=通路, 0=壁
        self.start = (0, 0)
        self.goal = (height-1, width-1)
        self.dynamic_obstacles = []
        self.multiple_goals = []
        
        self._generate_maze()
    
    def _generate_maze(self):
        """迷路生成"""
        if self.maze_type == "simple":
            self._generate_simple_maze()
        elif self.maze_type == "complex":
            self._generate_complex_maze()
        elif self.maze_type == "dynamic":
            self._generate_dynamic_maze()
        elif self.maze_type == "multigoal":
            self._generate_multigoal_maze()
    
    def _generate_simple_maze(self):
        """シンプル迷路生成"""
        # ランダムに壁を配置
        wall_density = 0.3
        for i in range(self.height):
            for j in range(self.width):
                if random.random() < wall_density:
                    self.grid[i, j] = 0
        
        # スタートとゴールは通路に
        self.grid[self.start] = 1
        self.grid[self.goal] = 1
        
        # 経路が存在することを保証
        self._ensure_path_exists()
    
    def _generate_complex_maze(self):
        """複雑迷路生成"""
        # より複雑なパターン
        wall_density = 0.4
        
        # パターン1: 迷路状の構造
        for i in range(0, self.height, 2):
            for j in range(self.width):
                self.grid[i, j] = 0
        
        for i in range(self.height):
            for j in range(0, self.width, 3):
                self.grid[i, j] = 0
        
        # ランダムな開口部
        for _ in range(self.width * self.height // 10):
            i, j = random.randint(0, self.height-1), random.randint(0, self.width-1)
            self.grid[i, j] = 1
        
        self.grid[self.start] = 1
        self.grid[self.goal] = 1
        self._ensure_path_exists()
    
    def _generate_dynamic_maze(self):
        """動的障害物迷路"""
        self._generate_simple_maze()
        
        # 動的障害物の追加
        num_dynamic = min(5, self.width * self.height // 20)
        for _ in range(num_dynamic):
            pos = (random.randint(0, self.height-1), random.randint(0, self.width-1))
            if pos != self.start and pos != self.goal:
                self.dynamic_obstacles.append(pos)
    
    def _generate_multigoal_maze(self):
        """複数ゴール迷路"""
        self._generate_simple_maze()
        
        # 複数ゴールの設定
        num_goals = min(3, max(2, self.width * self.height // 30))
        for _ in range(num_goals):
            goal_pos = (random.randint(0, self.height-1), random.randint(0, self.width-1))
            if goal_pos != self.start and self.grid[goal_pos] == 1:
                self.multiple_goals.append(goal_pos)
        
        # 元のゴールも含める
        if self.goal not in self.multiple_goals:
            self.multiple_goals.append(self.goal)
    
    def _ensure_path_exists(self):
        """経路存在を保証（DFSで接続性確認）"""
        visited = set()
        stack = [self.start]
        
        while stack:
            pos = stack.pop()
            if pos in visited:
                continue
            visited.add(pos)
            
            if pos == self.goal:
                return  # 経路存在
            
            for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
                ni, nj = pos[0] + di, pos[1] + dj
                if (0 <= ni < self.height and 0 <= nj < self.width and 
                    self.grid[ni, nj] == 1 and (ni, nj) not in visited):
                    stack.append((ni, nj))
        
        # 経路が存在しない場合、強制的に作成
        self._create_path()
    
    def _create_path(self):
        """強制的に経路作成"""
        # シンプルな直線経路を作成
        i, j = self.start
        while (i, j) != self.goal:
            self.grid[i, j] = 1
            if i < self.goal[0]:
                i += 1
            elif j < self.goal[1]:
                j += 1
    
    def is_valid_position(self, pos: Tuple[int, int], step: int = 0) -> bool:
        """位置の有効性チェック"""
        i, j = pos
        if not (0 <= i < self.height and 0 <= j < self.width):
            return False
        if self.grid[i, j] == 0:
            return False
        
        # 動的障害物チェック
        if self.maze_type == "dynamic":
            for obs_pos in self.dynamic_obstacles:
                # 時間によって動く障害物（簡単な例）
                moving_pos = ((obs_pos[0] + step // 5) % self.height, obs_pos[1])
                if pos == moving_pos:
                    return False
        
        return True
    
    def get_neighbors(self, pos: Tuple[int, int], step: int = 0) -> List[Tuple[int, int]]:
        """隣接する有効な位置を取得"""
        neighbors = []
        for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
            new_pos = (pos[0] + di, pos[1] + dj)
            if self.is_valid_position(new_pos, step):
                neighbors.append(new_pos)
        return neighbors
    
    def is_goal(self, pos: Tuple[int, int]) -> bool:
        """ゴール判定"""
        if self.maze_type == "multigoal":
            return pos in self.multiple_goals
        return pos == self.goal
    
    def visualize(self, path: List[Tuple[int, int]] = None, title: str = ""):
        """迷路可視化"""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # 迷路描画
        maze_visual = self.grid.copy().astype(float)
        
        # 壁を黒、通路を白に
        maze_visual = 1 - maze_visual  # 反転
        
        # スタートを緑、ゴールを赤に
        maze_visual = np.stack([maze_visual, maze_visual, maze_visual], axis=2)
        maze_visual[self.start] = [0, 1, 0]  # 緑
        
        if self.maze_type == "multigoal":
            for goal in self.multiple_goals:
                maze_visual[goal] = [1, 0, 0]  # 赤
        else:
            maze_visual[self.goal] = [1, 0, 0]  # 赤
        
        # 経路を青で描画
        if path:
            for pos in path[1:-1]:  # スタートとゴール以外
                maze_visual[pos] = [0, 0, 1]  # 青
        
        # 動的障害物を紫で描画
        if self.maze_type == "dynamic":
            for obs_pos in self.dynamic_obstacles:
                maze_visual[obs_pos] = [0.5, 0, 0.5]  # 紫
        
        ax.imshow(maze_visual)
        ax.set_title(f"{title}\n{self.maze_type} maze ({self.width}x{self.height})")
        ax.set_xticks([])
        ax.set_yticks([])
        
        return fig

print("✅ 迷路環境クラス実装完了")

In [ ]:
# 🧩 Phase 3: GEDIG Maze - Large Scale GPU Experiment

このノートブックでは、GPU最適化されたColab環境で、GEDIG（Graph Edit Distance + Information Gain）迷路解法アルゴリズムの大規模実験を行います。

## 🎯 実験目標
- **試行回数削減**: 60%削減
- **収束速度**: 3倍高速収束  
- **成功率**: 95%達成

複数のアルゴリズム（A*, Dijkstra, Q-Learning, 遺伝的アルゴリズム、SlimeMold_GEDIG）を比較し、GEDIG手法の優位性を実証します。